# Train an AutoEncoder to Reduce the Size of TF-IDF Vectors

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import src.models.cdea.CDAE as CDAE
import src.models.cdea.load_data as load_data
import src.models.cdea.metrics as metrics

from sklearn.metrics import precision_recall_fscore_support
import numpy as np

Using TensorFlow backend.


In [2]:
batch_size = 4
epochs = 10
embedding_size = 10
num_profiles = 10

In [3]:
# Load the proejct data
train_projects, train_x, test_projects, test_x, train_project_ids, test_project_ids = load_data.load_projects()
train_x_projects = np.array(train_projects, dtype=np.int32).reshape(len(train_projects), 1)
test_x_projects = np.array(test_projects, dtype=np.int32).reshape(len(test_projects), 1)

In [17]:
len(train_projects)

1424

In [15]:
train_x.shape

(1424, 13572)

In [16]:
train_x_projects.shape

(1424, 1)

In [4]:
# Create our model
model = CDAE.create(I=train_x.shape[1], U=len(train_projects)+1, K=embedding_size,
                    hidden_activation='relu', output_activation='sigmoid', q=0.50, l=0.01)
model.compile(loss='mean_absolute_error', optimizer='adam')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_item (InputLayer)             (None, 13572)        0                                            
__________________________________________________________________________________________________
x_user (InputLayer)             (None, 1)            0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 13572)        0           x_item[0][0]                     
__________________________________________________________________________________________________
embedding_layer (Embe

/Users/thomascartwright/Documents/Development/sci-autoencoder/src/models/cdea/CDAE.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_regularizer=<keras.reg..., bias_regularizer=<keras.reg...)`
  h_item = Dense(K, W_regularizer=l2(l), b_regularizer=l2(l))(h_item)
/Users/thomascartwright/Documents/Development/sci-autoencoder/src/models/cdea/CDAE.py:29: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(input_dim=1425, output_dim=10, input_length=1, name="embedding_layer", embeddings_regularizer=<keras.reg...)`
  h_user = Embedding(input_dim=U, output_dim=K, input_length=1, W_regularizer=l2(l), name='embedding_layer')(x_user)
/Users/thomascartwright/Documents/Development/sci-autoencoder/src/models/cdea/CDAE.py:39: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  return Model(input=[x_item, x_user], output=y)


In [5]:
train_x.shape

(1424, 13572)

In [12]:
train_x_projects[:90]

array([[ 0],
       [ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10],
       [11],
       [12],
       [13],
       [14],
       [15],
       [16],
       [17],
       [18],
       [19],
       [20],
       [21],
       [22],
       [23],
       [24],
       [25],
       [26],
       [27],
       [28],
       [29],
       [30],
       [31],
       [32],
       [33],
       [34],
       [35],
       [36],
       [37],
       [38],
       [39],
       [40],
       [41],
       [42],
       [43],
       [44],
       [45],
       [46],
       [47],
       [48],
       [49],
       [50],
       [51],
       [52],
       [53],
       [54],
       [55],
       [56],
       [57],
       [58],
       [59],
       [60],
       [61],
       [62],
       [63],
       [64],
       [65],
       [66],
       [67],
       [68],
       [69],
       [70],
       [71],
       [72],
       [73],
       [74],
       [75],
       [76],

In [6]:
# Train our Autoencoder
history = model.fit(x=[train_x, train_x_projects], y=train_x,
                    batch_size=batch_size, nb_epoch=epochs, verbose=1,
#                    validation_split=0.2)
                    validation_data=[[test_x, test_x_projects], test_x])

/Users/thomascartwright/miniconda3/envs/cdea/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Instructions for updating:
Use tf.cast instead.
Train on 1424 samples, validate on 357 samples
Epoch 1/10
1424/1424 [==============================] - 3s 2ms/step - loss: 0.3234 - val_loss: 0.1217
Epoch 2/10
1424/1424 [==============================] - 3s 2ms/step - loss: 0.0779 - val_loss: 0.0617
Epoch 3/10
1424/1424 [==============================] - 3s 2ms/step - loss: 0.0537 - val_loss: 0.0486
Epoch 4/10
1424/1424 [==============================] - 3s 2ms/step - loss: 0.0437 - val_loss: 0.0404
Epoch 5/10
1076/1424 [=====================>........] - ETA: 0s - loss: 0.0375

KeyboardInterrupt: 

In [ ]:
# Save the model
model.save('autoencoder.h5')